In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive/Colab/DynAE')
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
from DynAE import DynAE
from datasets import load_data
import metrics

In [0]:
dataset ='mnist'
loss_weight_lambda=0.5
save_dir='/content/drive/My Drive/Colab/DynAE/results'
visualisation_dir='/content/drive/My Drive/Colab/DynAE/visualisation'
data_path ='/content/drive/My Drive/Colab/data/' + dataset
batch_size=256
maxiter_pretraining=130e3
maxiter_clustering=1e5
tol=0.01
optimizer1=SGD(0.001, 0.9)
optimizer2=tf.train.AdamOptimizer(0.0001)
kappa = 3
ws=0.1
hs=0.1
rot=10
scale=0.
gamma=100

In [0]:
x, y = load_data(dataset, data_path)
n_clusters = len(np.unique(y))
model = DynAE(batch_size=batch_size, dataset=dataset, dims=[x.shape[-1], 500, 500, 2000, 10], loss_weight=loss_weight_lambda, gamma=gamma, n_clusters=n_clusters, visualisation_dir=visualisation_dir, ws=ws, hs=hs, rot=rot, scale=scale)
model.compile_dynAE(optimizer=optimizer1)
model.compile_disc(optimizer=optimizer2)
model.compile_aci_ae(optimizer=optimizer2)

In [0]:
#Load the pretraining weights if you have already pretrained your network

model.ae.load_weights(save_dir + '/' + dataset + '/pretrain/ae_weights.h5')
#model.critic.load_weights(save_dir + '/' + dataset + '/pretrain/critic_weights.h5')

In [0]:
#Pretraining phase

model.train_aci_ae(x, y, maxiter=maxiter_pretraining, batch_size=batch_size, validate_interval=2800, save_interval=2800, save_dir=save_dir, verbose=1, aug_train=True)

In [0]:
#Save the pretraining weights if you do not want to pretrain your model again

#model.ae.save_weights(save_dir + '/' + dataset + '/pretrain/ae_weights.h5')
#model.critic.save_weights(save_dir + '/' + dataset + '/pretrain/critic_weights.h5')

In [0]:
#Clustering phase

y_pred = model.train_dynAE(x=x, y=y, kappa=kappa, n_clusters=n_clusters, maxiter=maxiter_clustering, batch_size=batch_size, tol=tol, validate_interval=140, show_interval=None, save_interval=2800, save_dir=save_dir, aug_train=True)

In [0]:
#Save the clustering weights

model.ae.save_weights(save_dir + '/' + dataset + '/cluster/ae_weights.h5')

In [0]:
#Print ACC and NMI

model.compute_acc_and_nmi(x, y)